In [1]:
import subprocess as sp
import os
import ffmpeg
import numpy as np
import cv2
from PIL import Image

In [40]:
workdir = os.getcwd()
video_path = workdir + '/test.mp4'
out_filename = workdir + '/frames/out.mp4'

probe = ffmpeg.probe(video_path)
video_info = next(x for x in probe['streams'] if x['codec_type'] == 'video')
width = int(video_info['width'])
height = int(video_info['height'])
num_frames = int(video_info['nb_frames'])

In [41]:
print(video_info)

{'index': 0, 'codec_name': 'h264', 'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10', 'profile': 'Main', 'codec_type': 'video', 'codec_tag_string': 'avc1', 'codec_tag': '0x31637661', 'width': 1920, 'height': 1080, 'coded_width': 1920, 'coded_height': 1080, 'closed_captions': 0, 'has_b_frames': 1, 'sample_aspect_ratio': '1:1', 'display_aspect_ratio': '16:9', 'pix_fmt': 'yuvj420p', 'level': 40, 'color_range': 'pc', 'chroma_location': 'left', 'refs': 1, 'is_avc': 'true', 'nal_length_size': '4', 'r_frame_rate': '241/12', 'avg_frame_rate': '4597500/229091', 'time_base': '1/30000', 'start_pts': 0, 'start_time': '0.000000', 'duration_ts': 916364, 'duration': '30.545467', 'bit_rate': '12113230', 'bits_per_raw_sample': '8', 'nb_frames': '613', 'disposition': {'default': 1, 'dub': 0, 'original': 0, 'comment': 0, 'lyrics': 0, 'karaoke': 0, 'forced': 0, 'hearing_impaired': 0, 'visual_impaired': 0, 'clean_effects': 0, 'attached_pic': 0, 'timed_thumbnails': 0}, 'tags': {'creation_time':

In [42]:
# out, err = (ffmpeg
#         .input(video_path)
#         .output(workdir + '/frames/%04d.jpg')
#         .run(capture_stdout=True)
# )

In [58]:
def ffmpeg_reading_process(filename):
    args = (
        ffmpeg
        .input(filename)
        .output('pipe:', crf=0, preset='ultrafast', format='rawvideo', pix_fmt='rgb24')
        .compile()
    )
    return sp.Popen(args, stdout=sp.PIPE)

def ffmpeg_writing_process(filename, width, height):
    args = (
        ffmpeg
        .input('pipe:', format='rawvideo', pix_fmt='rgb24', s='{}x{}'.format(width, height))
        .filter('fps', fps=30, round='up')
        .drawtext(text="Hello",
            fontcolor="red",x=40,y=100,
            fontsize="64")
        .setpts('1.2*PTS')
        .output(filename, crf=0, preset='ultrafast', movflags='faststart', pix_fmt='rgb24')
        .overwrite_output()
        .compile()
    )
    return sp.Popen(args, stdin=sp.PIPE)

def read_frame(process, width, height):
    frame_size = width * height * 3
    in_bytes = process.stdout.read(frame_size)
    if len(in_bytes) == 0:
        frame = None
    else:
        assert len(in_bytes) == frame_size
        frame = (
            np
            .frombuffer(in_bytes, np.uint8)
            .reshape([height, width, 3])
        )
    return frame

def write_frame(process, frame):
       
    process.stdin.write(
        frame
        .astype(np.uint8)
        .tobytes()
    )

In [59]:
process_read = ffmpeg_reading_process(video_path)
process_write = ffmpeg_writing_process(out_filename, width, height)

while True:
    frame = read_frame(process_read, width, height)
    
    if frame is not None:
        write_frame(process_write, frame)
    else:
        break

process_read.wait()
print("Process read done.")
process_write.stdin.close()
process_write.wait()
print("Process write done.")

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Process read done.
Process write done.


frame=  884 fps=100 q=-1.0 Lsize=  550617kB time=00:00:29.43 bitrate=153249.4kbits/s dup=147 drop=0 speed=3.34x    
video:550606kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.001945%
[libx264 @ 0x5568d3053640] frame I:4     Avg QP: 0.00  size:1024902
[libx264 @ 0x5568d3053640] frame P:880   Avg QP: 0.00  size:636046
[libx264 @ 0x5568d3053640] mb I  I16..4: 100.0%  0.0%  0.0%
[libx264 @ 0x5568d3053640] mb P  I16..4: 29.1%  0.0%  0.0%  P16..4: 39.4%  0.0%  0.0%  0.0%  0.0%    skip:31.5%
[libx264 @ 0x5568d3053640] coded y,u,v intra: 53.6% 52.4% 47.8% inter: 49.3% 41.6% 36.6%
[libx264 @ 0x5568d3053640] i16 v,h,dc,p: 76% 24%  0%  0%
[libx264 @ 0x5568d3053640] kb/s:153073.30


In [ ]:
# process_read = ffmpeg_reading_process(video_path)

# while True:
#     frame = read_frame(process_read, width, height)
#     img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     cv2.imshow("Image", img)
#     k = cv2.waitKey(0)
#     if k == 27:         # wait for ESC key to exit
#         break
    
#     process.stdout.flush()
        
# cv2.destroyAllWindows()
# process.terminate()